In [1]:
import numpy as np
from numpy import float32
import warnings
import os
import shutil
import sys
import glob
import tensorflow as tf
import cv2
from sklearn import utils
from sklearn import preprocessing
import matplotlib.pyplot as plt
import Augmentor
from pathlib import Path
import astropy.stats

import tensorflow.keras.backend as K
from tensorflow.keras.applications import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.preprocessing import *
from tensorflow.keras.utils import *
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

sys.path.append(os.path.join(Path.cwd(), 'utils'))

from utils.im_utils import *
from utils.resnet_model import *
from utils.simple_conv_model import *

In [2]:
# os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
FLARE_CLASS = 'M'

TRAIN_DATA_DIR = f'./data/{FLARE_CLASS}_data/train'
VAL_DATA_DIR = f'./data/{FLARE_CLASS}_data/val'
TEST_DATA_DIR = f'./data/{FLARE_CLASS}_data/test'

AUG_TRAIN_DATA_DIR = f'./data/{FLARE_CLASS}_data_augmented/train'
AUG_VAL_DATA_DIR = f'./data/{FLARE_CLASS}_data_augmented/val'
AUG_TEST_DATA_DIR = f'./data/{FLARE_CLASS}_data_augmented/test'

AUG_PAIR_TRAIN_DATA_DIR = f'./data/{FLARE_CLASS}_data_augmented_pair/train'
AUG_PAIR_VAL_DATA_DIR = f'./data/{FLARE_CLASS}_data_augmented_pair/val'
AUG_PAIR_TEST_DATA_DIR = f'./data/{FLARE_CLASS}_data_augmented_pair/test'

# PAIR_TRAIN_DATA_DIR = f'./data/{FLARE_CLASS}_full_image_data/train'
# PAIR_VAL_DATA_DIR = f'./data/{FLARE_CLASS}_full_image_data/val'
# PAIR_TEST_DATA_DIR = f'./data/{FLARE_CLASS}_full_image_data/test'

# AUG_PAIR_TRAIN_DATA_DIR = f'./data/{FLARE_CLASS}_full_image_data_augmented/train'
# AUG_PAIR_VAL_DATA_DIR = f'./data/{FLARE_CLASS}_full_image_data_augmented/val'
# AUG_PAIR_TEST_DATA_DIR = f'./data/{FLARE_CLASS}_full_image_data_augmented/test'

RESNET_CHECKPOINTS_DIR = './checkpoints/resnet_checkpoints'

In [4]:
def delete_files(folder):
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

In [5]:
def augment_train_data():
    delete_files(f'{AUG_TRAIN_DATA_DIR}/positive')
    delete_files(f'{AUG_TRAIN_DATA_DIR}/negative')
    
    for subdir, dirs, files in os.walk(TRAIN_DATA_DIR):
        for file in files:
            filepath = os.path.join(subdir, file)
            img = np.load(filepath)
            # rotated_imgs = [rotate_img(img, 90), rotate_img(img, 180), rotate_img(img, 270)]
            # for idx, rot_img in enumerate(rotated_imgs):
            #     rotated_img_save = os.path.join(subdir, f'{file}_{idx}_rot')
            #     np.save(rotated_img_save, rot_img)

            fliplr_img = np.fliplr(img)
            flipud_img = np.flipud(img)
            rot90_img = np.rot90(img)
            rot180_img = np.rot90(img, 2)
            rot270_img = np.rot90(img, 3)
            
            folder = subdir.rsplit('/', 1)[1]
            cut_filename = file.rsplit('.', 1)[0]

            original_img_save = os.path.join(f'{AUG_TRAIN_DATA_DIR}/{folder}', cut_filename)
            fliplr_img_save = os.path.join(f'{AUG_TRAIN_DATA_DIR}/{folder}', f'{cut_filename}_fliplr')
            flipud_img_save = os.path.join(f'{AUG_TRAIN_DATA_DIR}/{folder}', f'{cut_filename}_flipud')
            rot90_img_save = os.path.join(f'{AUG_TRAIN_DATA_DIR}/{folder}', f'{cut_filename}_rot90')
            rot180_img_save = os.path.join(f'{AUG_TRAIN_DATA_DIR}/{folder}', f'{cut_filename}_rot180')
            rot270_img_save = os.path.join(f'{AUG_TRAIN_DATA_DIR}/{folder}', f'{cut_filename}_rot270')

            np.save(original_img_save, img)
            np.save(fliplr_img_save, fliplr_img)
            np.save(flipud_img_save, flipud_img)
            np.save(rot90_img_save, rot90_img)
            np.save(rot180_img_save, rot180_img)
            np.save(rot270_img_save, rot270_img)

            # translated_img = translate(img)
            # translated_img_save = os.path.join(subdir, f'{file}_trans')
            # np.save(translated_img_save, translated_img)

In [6]:
def augment_pair_train_data(data_category):
    data_path = f'{PAIR_TRAIN_DATA_DIR}/{data_category}'
    aug_data_path = f'{AUG_PAIR_TRAIN_DATA_DIR}/{data_category}'
    delete_files(aug_data_path)
    # delete_files(f'{AUG_TRAIN_DATA_DIR}/negative')
    
    for subdir, dirs, files in os.walk(data_path):
        for file in files:
            filepath = os.path.join(subdir, file)
            file_folder = filepath.rsplit('/')[-2]
            original_img_sequence_folder_path = f'{aug_data_path}/{file_folder}'
            fliplr_img_sequence_folder_path = f'{aug_data_path}/{file_folder}_fliplr'
            flipud_img_sequence_folder_path = f'{aug_data_path}/{file_folder}_flipud'
            rot90_img_sequence_folder_path = f'{aug_data_path}/{file_folder}_rot90'
            rot180_img_sequence_folder_path = f'{aug_data_path}/{file_folder}_rot180'
            rot270_img_sequence_folder_path = f'{aug_data_path}/{file_folder}_rot270'
            
            if not os.path.exists(original_img_sequence_folder_path):
                os.makedirs(original_img_sequence_folder_path)
            if not os.path.exists(fliplr_img_sequence_folder_path):
                os.makedirs(fliplr_img_sequence_folder_path)
            if not os.path.exists(flipud_img_sequence_folder_path):
                os.makedirs(flipud_img_sequence_folder_path)
            if not os.path.exists(rot90_img_sequence_folder_path):
                os.makedirs(rot90_img_sequence_folder_path)
            if not os.path.exists(rot180_img_sequence_folder_path):
                os.makedirs(rot180_img_sequence_folder_path)
            if not os.path.exists(rot270_img_sequence_folder_path):
                os.makedirs(rot270_img_sequence_folder_path)
            
            img = np.load(filepath)
            fliplr_img = np.fliplr(img)
            flipud_img = np.flipud(img)
            rot90_img = np.rot90(img)
            rot180_img = np.rot90(img, 2)
            rot270_img = np.rot90(img, 3)
            cut_filename = file.rsplit('.', 1)[0]
            
            original_img_save = os.path.join(original_img_sequence_folder_path, cut_filename)
            fliplr_img_save = os.path.join(fliplr_img_sequence_folder_path, f'{cut_filename}')
            flipud_img_save = os.path.join(flipud_img_sequence_folder_path, f'{cut_filename}')
            rot90_img_save = os.path.join(rot90_img_sequence_folder_path, f'{cut_filename}')
            rot180_img_save = os.path.join(rot180_img_sequence_folder_path, f'{cut_filename}')
            rot270_img_save = os.path.join(rot270_img_sequence_folder_path, f'{cut_filename}')

            np.save(original_img_save, img)
            np.save(fliplr_img_save, fliplr_img)
            np.save(flipud_img_save, flipud_img)
            np.save(rot90_img_save, rot90_img)
            np.save(rot180_img_save, rot180_img)
            np.save(rot270_img_save, rot270_img)

In [7]:
# augment_train_data()

In [8]:
# augment_pair_train_data('positive')
# augment_pair_train_data('negative')

In [9]:
def sigma_clip(image, mul=3):
    scs = astropy.stats.sigma_clipped_stats(image)
    sd = scs[2]
    thr = sd*mul
    image[image<thr] = 0
    
    return image

In [10]:
class CustomDataGen(tf.keras.utils.Sequence):
    
    def __init__(self, files_paths,
                 batch_size,
                 input_size=(64, 64, 1),
                 shuffle=True):
        
        self.files_paths = files_paths.copy()
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        
        self.n = len(self.files_paths)
        self.n_category = 2
        # self.n_name = df[y_col['name']].nunique()
        # self.n_type = df[y_col['type']].nunique()
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.files_paths)
    
    def __getitem__(self, index):
        batches = self.files_paths[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__get_data(batches)        
        return X, y
    
    def __len__(self):
        return self.n // self.batch_size
    
    def __get_input(self, path):
        # image = preprocessing.normalize(np.load(path))
        image = np.load(path)
        image = np.expand_dims(image, axis=2)
        # image = NormalizeData(image)
        image = cv2.resize(image, (128, 128), interpolation = cv2.INTER_AREA)
        scs = astropy.stats.sigma_clipped_stats(image)
        # sd = scs[2]
        # thr = sd*5
        image[image<0] = 0
        # image = preprocessing.normalize(image)
        return image

    def __get_output(self, path, num_classes=2):
        label = None
        folder = path.rsplit('/')[-2]
        if folder == 'positive':
            label = 1
        elif folder == 'negative':
            label = 0
        
        return label
        # return tf.keras.utils.to_categorical(label, num_classes=num_classes)
    
    def __get_data(self, batches):
        # Generates data containing batch_size samples
        # path_batch = batches[self.X_col['path']]
        # category_batch = batches[self.y_col['type']]

        X_batch = np.asarray([self.__get_input(x) for x in batches])
        y_batch = np.asarray([self.__get_output(y, self.n_category) for y in batches])

        return X_batch, y_batch

In [11]:
class CustomPairDataGen(tf.keras.utils.Sequence):
    
    def __init__(self, folder_paths,
                 batch_size,
                 shuffle=True):
        
        self.folder_paths = folder_paths.copy()
        self.batch_size = batch_size
        self.shuffle = shuffle
        
        self.n = len(self.folder_paths)
        self.n_category = 2
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.folder_paths)
    
    def __getitem__(self, index):
        batches = self.folder_paths[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__get_data(batches)  
        X = np.expand_dims(X, axis=4)
        return X, y
    
    def __len__(self):
        return self.n // self.batch_size
    
    def __get_input(self, folder):
        images = []
        for subdir, dirs, files in os.walk(folder):
            for f in files:
                images.append(os.path.join(subdir, f.decode(encoding='UTF-8')))
        images = sorted(images)
        images = [np.load(x) for x in images]
        if len(images) != 2:
            print(folder)
        images = [cv2.resize(x, (128, 128), interpolation = cv2.INTER_AREA) for x in images]
        images = [sigma_clip(x) for x in images]
        images  = np.array(images)
        return images

    def __get_output(self, path, num_classes=2):
        label = None
        folder = path.rsplit('/')[-2]
        if folder == 'positive':
            label = 1
        elif folder == 'negative':
            label = 0
        
        return label
    
    def __get_data(self, batches):
        X_batch = np.asarray([self.__get_input(x) for x in batches])
        y_batch = np.asarray([self.__get_output(y, self.n_category) for y in batches])

        return X_batch, y_batch

In [12]:
def GetDataPaths(train_data_dir, val_data_dir, test_data_dir):
    train_files = []
    for subdir, dirs, files in os.walk(train_data_dir):
         for f in files:
            train_files.append(os.path.join(subdir, f))
    train_files = np.array(train_files)

    val_files = []
    for subdir, dirs, files in os.walk(val_data_dir):
         for f in files:
            val_files.append(os.path.join(subdir, f))
    val_files = np.array(val_files)

    test_files = []
    for subdir, dirs, files in os.walk(test_data_dir):
         for f in files:
            test_files.append(os.path.join(subdir, f))
    test_files = np.array(test_files)
    
    return train_files, val_files, test_files

In [13]:
def GetDataFolders(train_data_dir, val_data_dir, test_data_dir):
    train_folders = []
    for subdir, dirs, files in os.walk(train_data_dir):
        for d in dirs:
            if d != 'positive' and d != 'negative' and d != '.ipynb_checkpoints':
                train_folders.append(os.path.join(subdir, d))
    train_folders = np.array(train_folders)

    val_folders = []
    for subdir, dirs, files in os.walk(val_data_dir):
         for d in dirs:
                if d != 'positive' and d != 'negative' and d != '.ipynb_checkpoints':
                    val_folders.append(os.path.join(subdir, d))
    val_folders = np.array(val_folders)

    test_folders = []
    for subdir, dirs, files in os.walk(test_data_dir):
         for d in dirs:
                if d != 'positive' and d != 'negative' and d != '.ipynb_checkpoints':
                    test_folders.append(os.path.join(subdir, d))
    test_folders = np.array(test_folders)
    
    return train_folders, val_folders, test_folders

In [14]:
def get_test_conv_model():
    inp = Input(shape=(64, 64, 1))
    x = Conv2D(filters=48, kernel_size=4)(inp)
    x = MaxPooling2D(pool_size=(3, 3))(x)
    x = Conv2D(filters=24, kernel_size=3)(x)
    x = MaxPooling2D(pool_size=(3, 3))(x)
    x = Conv2D(filters=12, kernel_size=3)(x)
    x = MaxPooling2D(pool_size=(3, 3))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(1, activation='sigmoid')(x)
    
    model = Model(inp, x)
    adam_fine = Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999, decay=0.0002, amsgrad=False)
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000,
    decay_rate=0.9)
    optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
    model.compile(
        loss="binary_crossentropy", optimizer=adam_fine, metrics=["accuracy"]
    )
    
    return model

In [15]:
def CustomAlexNet1():
    inp = Input(shape=(128, 128, 1))
    x = Conv2D(filters=96, kernel_size=(11, 11), activation='relu')(inp)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(3, 3))(x)
    
    x = Conv2D(filters=256, kernel_size=(5, 5), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(3, 3))(x)
    
    x = Conv2D(filters=384, kernel_size=(3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(filters=384, kernel_size=(3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    
    x = MaxPool2D(pool_size=(3, 3))(x)
    
    x = Flatten()(x)
    
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    # x = Dense(1, activation='sigmoid')(x)
    
    model = Model(inp, x)
    
    return model

In [17]:
def CustomPairAlexNet():
    inp = Input(shape=(2, 128, 128, 1))
    cutout_inp = tf.convert_to_tensor([inp[x][0] for x in range(256)])
    full_inp = tf.convert_to_tensor([inp[x][1] for x in range(256)])
    print(cutout_inp.shape)
    print(full_inp.shape)
    # full_inp = Input(shape=(128, 128, 1))
    
    cutout_features = CustomAlexNet1()(cutout_inp)
    full_features = CustomAlexNet1()(full_inp)
    
    x = concatenate([cutout_features, full_features])
    x = Dense(1, activation='sigmoid')(x)
    
    model = Model(inp, x)
    
    return model

In [18]:
# model = CustomAlexNet1()
model = CustomPairAlexNet()

2022-08-15 10:10:50.385055: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-15 10:10:50.902944: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-15 10:10:50.905275: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-15 10:10:50.906942: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

(256, 128, 128, 1)
(256, 128, 128, 1)


In [26]:
batch_size = 256
epochs=30
train_files, val_files, test_files = GetDataFolders(AUG_PAIR_TRAIN_DATA_DIR, AUG_PAIR_VAL_DATA_DIR, AUG_PAIR_TEST_DATA_DIR)

In [30]:
traingen = CustomPairDataGen(train_files, batch_size)
valgen = CustomPairDataGen(val_files, batch_size)

val_x = np.concatenate([valgen[x][0] for x in range(len(valgen)+1)])
val_y = np.concatenate([valgen[x][1] for x in range(len(valgen)+1)])

In [41]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

adam_fine = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, decay=0.0002, amsgrad=False)
model.compile(loss="binary_crossentropy", optimizer=adam_fine, metrics=["accuracy"])
history = model.fit(traingen, validation_data = valgen, epochs=epochs, batch_size=batch_size)

Epoch 1/30
182/182 [==============================] - 177s 950ms/step - loss: 0.8873 - accuracy: 0.7335 - val_loss: 4.8535 - val_accuracy: 0.5273
Epoch 2/30
182/182 [==============================] - 169s 927ms/step - loss: 0.3671 - accuracy: 0.8564 - val_loss: 0.4098 - val_accuracy: 0.8281
Epoch 3/30
182/182 [==============================] - 168s 920ms/step - loss: 0.2824 - accuracy: 0.8918 - val_loss: 0.7493 - val_accuracy: 0.7852
Epoch 4/30
182/182 [==============================] - 168s 921ms/step - loss: 0.1558 - accuracy: 0.9371 - val_loss: 1.1133 - val_accuracy: 0.7812
Epoch 5/30
182/182 [==============================] - 168s 919ms/step - loss: 0.0368 - accuracy: 0.9879 - val_loss: 1.8501 - val_accuracy: 0.7383
Epoch 6/30
182/182 [==============================] - 168s 920ms/step - loss: 0.0205 - accuracy: 0.9948 - val_loss: 1.5211 - val_accuracy: 0.8320
Epoch 7/30
182/182 [==============================] - 168s 922ms/step - loss: 0.0084 - accuracy: 0.9979 - val_loss: 1.5468 -

In [42]:
# model.save_weights(f'{RESNET_CHECKPOINTS_DIR}/alexnet_aug_pair')

In [19]:
model.load_weights(f'{RESNET_CHECKPOINTS_DIR}/alexnet_aug_pair')

In [ ]:
batch_size = 128
epochs=30
train_files, val_files, test_files = GetDataPaths(AUG_TRAIN_DATA_DIR, AUG_VAL_DATA_DIR, AUG_TEST_DATA_DIR)
traingen = CustomDataGen(train_files, batch_size)
valgen = CustomDataGen(val_files, batch_size)
testgen = CustomDataGen(test_files, batch_size)

In [ ]:
# train_x = np.concatenate([traingen[x][0] for x in range(len(traingen)+1)])
# train_y = np.concatenate([traingen[x][1] for x in range(len(traingen)+1)])

val_x = np.concatenate([valgen[x][0] for x in range(len(valgen)+1)])
val_y = np.concatenate([valgen[x][1] for x in range(len(valgen)+1)])

In [ ]:
# mirrored_strategy = tf.distribute.MirroredStrategy()
# with mirrored_strategy.scope():
#     model = CustomAlexNet()
#     adam_fine = Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, decay=0.0002, amsgrad=False)
#     model.compile(loss="binary_crossentropy", optimizer=adam_fine, metrics=["accuracy"])
#     history = model.fit(traingen, validation_data = valgen, epochs=epochs, batch_size=batch_size)

In [ ]:
# model = get_simple_conv_model(include_top=True)
# model = get_test_conv_model()
# model = CustomResNet50(include_top=True)
model = CustomAlexNet()
adam_fine = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, decay=0.0002, amsgrad=False)

# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=1e-4,
#     decay_steps=10000,
#     decay_rate=0.9)
# optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
    
model.compile(loss="binary_crossentropy", optimizer=adam_fine, metrics=["accuracy"])
# callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)]
# model.summary()
history = model.fit(traingen, validation_data = (val_x, val_y), epochs=epochs, batch_size=batch_size)

In [47]:
val_x.shape

(332, 2, 128, 128, 1)

In [45]:
val_x[0][0].shape

(128, 128, 1)

In [48]:
predictions = model.predict(valgen)
true_vals = val_y[:256]

correct = 0
for idx, pred in enumerate(predictions):
    pred_round = 0
    if pred >= 0.5:
        pred_round = 1
    if pred_round == true_vals[idx]:
        correct += 1

print(f'accuracy: {correct/(len(true_vals))}')

1/1 [==============================] - 1s 785ms/step
accuracy: 0.4140625


In [37]:
val_x.shape

(332, 2, 128, 128, 1)

In [34]:
predictions.shape

(256, 1)

In [ ]:
predictions[:20]

In [ ]:
# model.save_weights(f'{RESNET_CHECKPOINTS_DIR}/simple_conv_custom_norm')

In [ ]:
model.save_weights(f'{RESNET_CHECKPOINTS_DIR}/simple_conv_relu')

In [ ]:
epochs = 30
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
adam_fine = Adam(learning_rate=0.00005, beta_1=0.9, beta_2=0.999, decay=0.0002, amsgrad=False)
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

In [ ]:
model = CustomResNet50(include_top=True)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

history = model.fit(traingen, validation_data = valgen, epochs=epochs)

In [ ]:
model.save_weights(f'{RESNET_CHECKPOINTS_DIR}/sgd_newdata_B_class_1e-3_checkpoint')

In [ ]:
t = np.load('./data/B_data/train/positive/AIA20160107_0354_0094_0.npy')
t = cv2.resize(t, (128, 128), interpolation = cv2.INTER_AREA)
t = np.array([t.reshape(128, 128, 1)]) 

In [ ]:
model.predict(t)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()